# Pre-scaling: Develop scripts to enable scalable workflows and a clean notebook - Quynh
This task completion table summarizes the core scripts, updates, and components built to support Experiment 1 and Experiment 2 within a structured retrieval and benchmarking pipeline.

All code updates for the scripts have been fully encapsulated and organized to run cleanly within the Google Colab environment. This section focuses only on execution of the pipeline components, not the implementation details. To review the updated and newly created scripts, please visit the link (especially ./pipeline and ./scripts folders): https://drive.google.com/drive/folders/1CfiAaSM1y63JTdHkFwKuKruwFMRA1vaA?dmr=1&ec=wgc-drive-hero-goto

| **Pipeline Component**   | **Completed Task**            | **Description**                                                                                                                     |
| ------------------------ | ----------------------------- | ----------------------------------------------------------------------------------------------------------------------------------- |
| **Experiment 1: FAISS Single**         | `embedding_slice.py`          | Automated slice-embedding generation for wiki-30k, multi-50k, and future datasets.                                                  |
|                          | `build_faiss_index.py`        | Automated building of FAISS single index + generated matching config JSON.                                                          |
| **Experiment 1: FAISS Sharded**        | `shard_index_2.py`            | Updated sharding script to support embedding slicing, produce shard outputs, and build `manifest.json`.                             |
|                          | `build_faiss_index.py`        | Automated and consolidated config JSON format for FAISS Sharded + Spark fan-out (reusable).                                         |
|                                 | `inject_sharded_config.py`    | Generated and standardized sharded configuration files to ensure consistent setup across FAISS Sharded and Spark retrieval.         |
| **Experiment 1: Chroma**               | `chroma_ingest_v2.py`         | Updated ingestion script to load metadata, support multi-domain ingestion, and prepare Chroma indexes.                              |
|                          | `build_faiss_index.py`        | Automated configuration JSON generation for Chroma (reusable).                                                                      |
|                                 | `inject_chroma_config.py`    | Generated and standardized sharded configuration files to ensure consistent setup across FAISS Chroma retrieval.         |
| **Experiment 1: Spark Backend**        | `spark_backend.py`            | Created new Spark backend class at `pipeline/backends/spark_backend.py`.                                                            |
|                          | `spark_fanout.py` update      | Updated `run_spark_fanout()` in `scripts/spark_fanout.py` to correctly support sharded manifests.                                   |
|                          | `runner.py` update            | Registered Spark backend in the updated pipeline registry.                                                                          |
|                          | Config updates                | Added sharded index block in config.json (e.g., `multi_sharded`), added `"spark"` backend and `manifest_path` in extras.            |
|                          | `inject_spark_config.py`      | Automated and consolidated Spark configuration JSON generation.                                                                     |
| **Experiment 1: Execution** | `bench.py` overhaul           | Updated to support inference metrics for FAISS Single, FAISS Sharded, Chroma, and Spark; integrated `summary={}` and `run_bench()`. |
|                          | System inference benchmarking | Created measurement scripts for performance evaluation of the entire retrieval pipeline.                                            |
| **Experiment 2: Big LLM no RAG**         | `llm_no_rag.py`               | Script to benchmark system performance for **LLM without datastore** vs. **Multi-domain 50k** retrieval.                            |



#Experiment 1: Single Domain vs Multi-domain
We will compare Wiki vs. Multi-Domain (Wiki + PubMed + StackExchange) on 3 different datastore size (10k, 30k, and 50k) across 4 different retrieval types: FAISS single, FAISS Sharded, Chroma, and Spark.

Total number of experiments are 24 experiments.

## Import the Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"


/content/drive/MyDrive/AMS 560 PROJECT_SCALING


In [ ]:
!pwd

/content/drive/MyDrive/AMS 560 PROJECT_SCALING


In [ ]:
  !pip install -r requirements.txt
  !apt-get install tree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/61.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

Reading package lists... Done
^C


In [ ]:
import os, chromadb, glob, json
import numpy, pandas, pyarrow, faiss, pyspark
from sentence_transformers import SentenceTransformer

# Wiki 30k (Quynh)
Wiki 30k is provided as the reference example; all other datastore sizes follow the same workflow and logic described in this documentation.

# I. REBUILD CHUNKS (Wiki-30k)
This section describes how we reconstructed the corpus by re-chunking the original documents into uniformly sized text segments. The goal is to create consistent, embedding-ready units that support fair comparison across retrieval methods. We outline the chunk size from Sruthi's preprocessing data, and the output manifest structure used for downstream embedding and indexing.

In [ ]:
  !python scripts/embedding_slice.py \
  --emb_path data/embeddings/wikipedia/wiki_vectors.npy \
  --meta_path data/embeddings/wikipedia/wiki_meta.parquet \
  --out_dir data/embeddings/wikipedia \
  --n 30000 \
  --domain wiki


Loading metadata: data/embeddings/wikipedia/wiki_meta.parquet
Metadata rows available: 66337
Slicing first 30000 metadata rows...
Saved: data/embeddings/wikipedia/wiki_meta_30000.parquet
Loading embeddings: data/embeddings/wikipedia/wiki_vectors.npy
Embedding matrix shape: (66337, 384)
📘 Slicing first 30000 embedding rows...
Saved: data/embeddings/wikipedia/wiki_embeddings_30000.npy
Files created:
  - data/embeddings/wikipedia/wiki_meta_30000.parquet
  - data/embeddings/wikipedia/wiki_embeddings_30000.npy


# II. SYSTEM MEASUREMENT (Wiki-30k)
For each of the datastore size, we will measure the system for each of 4 retrieval types:
- Faiss Single
- Faiss Sharded
- Chroma Database
- Spark

## 1. FAISS SINGLE EXPERIMENT (Wiki-30k DONE)
This section evaluates retrieval performance using a single, monolithic FAISS index. All embeddings are stored in one FAISS structure on a single machine to establish a baseline for latency, and throughput. We measure how index type (HNSW) and hyperparameters affect speed/quality trade-offs, providing the foundation for comparing more scalable architectures.
### 1a. Build faiss (30k)
This writes out .faiss index files plus the manifest.json. We report build time, and index size to quantify preprocessing cost before retrieval workloads.

In [ ]:
!python scripts/build_faiss_index.py \
  --subset_dir data/embeddings/wikipedia \
  --index_group wikipedia \
  --index_name wiki_faiss_single_30k \
  --domain wiki \
  --out_config configs/index/wiki_30k_config.json


Metadata file: data/embeddings/wikipedia/wiki_meta_30000.parquet
Embeddings file: data/embeddings/wikipedia/wiki_embeddings_30000.npy
Metadata rows: 30000
Embedding matrix: (30000, 384)
Building FAISS HNSW index...
Indexed vectors: 30000
Saved FAISS index: indices/wikipedia/wiki_faiss_single_30k.faiss
Config written to: configs/index/wiki_30k_config.json
Done.


### 1b. Measure System with FAISS SINGLE Retrieval Type
This section evaluates end-to-end system performance using the single FAISS index as the retrieval backend. We benchmark query latency, QPS throughput. The results represent the non-sharded baseline, allowing us to compare how much performance improves or degrades when scaling to multi-shard or distributed designs.

In [ ]:
!python bench.py \
  --config configs/index/wiki_30k_config.json \
  --index wiki_faiss_single_30k \
  --backend faiss \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-23 22:15:05.268913: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 22:15:05.286668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763936105.308001    9334 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763936105.314462    9334 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763936105.330575    9334 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## 2. FAISS SHARDED EXPERIMENT (Wiki-30k)

FAISS-sharded runs all shard searches inside one Python process, sequentially or with local threads. This section introduces horizontal scaling by splitting the dataset into multiple FAISS shards, each hosted by a separate worker. Queries are broadcast across shards and merged using fan-out retrieval. We measure how sharding impacts latency, and throughput. This experiment demonstrates how FAISS behaves in a distributed environment compared to the single-index baseline.

In [ ]:
!python scripts/shard_index_v2.py \
  --meta_path data/embeddings/wikipedia/wiki_meta_30000.parquet \
  --emb_path data/embeddings/wikipedia/wiki_embeddings_30000.npy \
  --domain wiki \
  --out_meta_dir data/embeddings/wiki_sharded \
  --out_vectors_dir data/embeddings/wiki_sharded \
  --out_index_dir indices/wiki_sharded \
  --out_manifest_dir indices/wiki_sharded \
  --n_shards 8 \
  --save_vectors 1


Loaded 30,000 vectors (dim=384)
Normalizing vectors (L2)...
Sharding domain 'wiki': 100% 8/8 [00:06<00:00,  1.23it/s]
Manifest written: indices/wiki_sharded/wiki_30000_manifest.json
Total rows = 30000


Inject all FAISS shards into the datastore using the wiki_30k configuration. This registers every shard (stored in faiss_dir + meta_dir) under the index
name "wiki_faiss_sharded_30k" and marks it as the active retrieval backend.

In [ ]:
!python scripts/inject_sharded_index.py \
  --config_path configs/index/wiki_30k_config.json \
  --index_name wiki_faiss_sharded_30k \
  --faiss_dir indices/wiki_sharded \
  --meta_dir data/embeddings/wiki_sharded \
  --set_active


Successfully added FAISS-sharded entry to config.json
New index name: wiki_faiss_sharded_30k
FAISS shards: 8
META shards: 8
Manifest: indices/wiki_sharded/wiki_30000_manifest.json
Updated config: configs/index/wiki_30k_config.json


The following command runs the benchmarking pipeline using the sharded FAISS index (wiki_faiss_sharded_30k) and evaluates retrieval quality with an LLM:

In [ ]:
!python bench.py \
  --config configs/index/wiki_30k_config.json \
  --backend faiss \
  --index wiki_faiss_sharded_30k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 \
  --iters 3 \
  --warmup 1 \
  --results_dir results \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-23 22:35:22.083554: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 22:35:22.101750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763937322.122815   14679 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763937322.129247   14679 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763937322.145282   14679 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## 3. CHROMA EXPERIMENT (Wiki-30k)
This section evaluates retrieval using ChromaDB, testing its performance as an alternative vector store optimized for lightweight ingestion and flexible filtering. We ingest the same embeddings into Chroma, run retrieval queries, and measure latency, and throughput behavior. This comparison highlights how Chroma’s architecture differs from FAISS in practical workloads.

This command ingests the 30k Wikipedia embeddings into a Chroma collection, using the provided metadata and vector files. It writes the fully built Chroma datastore to `persist_dir` and registers it under the collection name `wiki_chroma_30k`.


In [ ]:
!python scripts/chroma_ingest_2.py \
  --meta_path data/embeddings/wikipedia/wiki_meta_30000.parquet \
  --vec_path data/embeddings/wikipedia/wiki_embeddings_30000.npy \
  --persist_dir indices/wiki_chroma/wiki_chroma_30k \
  --collection wiki_chroma_30k

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Ingesting 30,000 records into 'wiki_chroma_30k' @ indices/wiki_chroma/wiki_chroma_30k
  0% 0/30 [00:00<?, ?it/s]Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given
100% 30/30 [00:57<00:00,  1.91s/it]
✓ Manifest created: indices/wiki_chroma/wiki_chroma_30k/wiki_chroma_30k_manifest.json
✓ Done.


This command generates a Chroma index configuration file for the `wiki_chroma_30k` collection, pointing to its persisted datastore directory. It outputs the finalized config JSON and registers the collection under the specified index name for downstream retrieval experiments.


In [ ]:
!python scripts/inject_chroma_config.py \
  --output_config configs/index/config_wiki_chroma_30k.json \
  --index_name wiki_chroma_30k \
  --persist_dir indices/wiki_chroma/wiki_chroma_30k \
  --collection_name wiki_chroma_30k

✓ Created new Chroma config: configs/index/config_wiki_chroma_30k.json
✓ Added index: wiki_chroma_30k
✓ active_index = wiki_chroma_30k


This command benchmarks retrieval performance using the Chroma backend with the `wiki_chroma_30k` index and evaluates query quality using an LLM. It runs multiple iterations, saves results to the specified directory, and reports both latency metrics and LLM-based relevance judgments.


In [ ]:
!python bench.py \
--config configs/index/config_wiki_chroma_30k.json \
--backend chroma \
--index wiki_chroma_30k \
--qfile data/queries/all_sources_eval.txt --top_k 3 --iters 3 --warmup 1 --results_dir results/wiki_chroma_30k \
--enable_llm \
--llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-23 23:49:31.660147: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 23:49:31.677989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763941771.699464   46630 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763941771.706173   46630 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763941771.722365   46630 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## 4. SPARK EXPERIMENT (Wiki 30k)

This section measures retrieval and scaling behavior using a distributed Spark pipeline. FAISS or Chroma shards are accessed through Spark workers, and retrieval is orchestrated through distributed fan-out, merging, and ranking. We evaluate how Spark affects throughput, and latency when scaling to large datasets or multiple shards. This section demonstrates distributed retrieval performance at cluster scale.

This command creates a Spark-compatible index configuration by converting the sharded FAISS manifest into a Spark retrieval config. It outputs the final JSON file, allowing Spark workers to load and query the `wiki_spark_30000` shards in distributed experiments.


In [ ]:
!python scripts/inject_spark_config.py \
  --index_key wiki_spark_30000 \
  --manifest_path indices/wiki_sharded/wiki_30000_manifest.json \
  --out_path configs/index/config_wiki_spark_30k.json


✓ Spark config.json created
  index_key     : wiki_spark_30000
  manifest_path : indices/wiki_sharded/wiki_30000_manifest.json
  out_path      : configs/index/config_wiki_spark_30k.json


This command benchmarks distributed retrieval using the Spark backend with the `wiki_spark_30000` index. It evaluates performance and LLM-judged relevance across several iterations, using Spark workers to fan out and merge retrieval results.


In [ ]:
!python bench.py \
  --config configs/index/config_wiki_spark_30k.json \
  --backend spark \
  --index wiki_spark_30000 \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-24 00:32:47.143148: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-24 00:32:47.161238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763944367.182747   66788 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763944367.189307   66788 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763944367.206690   66788 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# SUMMARY:
All commands together represent the full retrieval workflow: slicing embeddings, ingesting them into FAISS, Chroma, or Spark-compatible formats, registering each index, and running benchmark evaluations with optional LLM scoring. Other datastore sizes follow the exact same sequence of steps and logic outlined in this documentation.

# MULTI-DOMAIN (50k) - Quynh

# I. REBUILD CHUNKS (MULTI DOMAIN - 50k)

In [ ]:
  !python scripts/embedding_slice.py \
  --emb_path data/embeddings/multi_default/multi_vectors.npy \
  --meta_path data/embeddings/multi_default/multi_meta.parquet \
  --out_dir data/embeddings/multi_domain \
  --n 50000 \
  --domain multidomain


Loading metadata: data/embeddings/multi_default/multi_meta.parquet
Metadata rows available: 79164
Slicing first 50000 metadata rows...
Saved: data/embeddings/multi_domain/multidomain_meta_50000.parquet
Loading embeddings: data/embeddings/multi_default/multi_vectors.npy
Embedding matrix shape: (79164, 384)
Slicing first 50000 embedding rows...
Saved: data/embeddings/multi_domain/multidomain_embeddings_50000.npy
Files created:
  - data/embeddings/multi_domain/multidomain_meta_50000.parquet
  - data/embeddings/multi_domain/multidomain_embeddings_50000.npy


# II. SYSTEM MEASUREMENT (MULTI DOMAIN - 50k)
For each of the datastore size, we will measure the system for each of 4 retrieval types:
- Faiss Single
- Faiss Sharded
- Chroma Database
- Spark

## 1. FAISS SINGLE EXPERIMENT (MULTI DOMAIN - 50k)
### 1a. Build faiss (50k)

In [ ]:
!python scripts/build_faiss_index.py \
  --subset_dir data/embeddings/multi_domain \
  --index_group wikipedia \
  --index_name multidomain_faiss_single_50k \
  --domain multidomain \
  --out_config configs/index/config_mutidomain_faiss_50k.json


Metadata file: data/embeddings/multi_domain/multidomain_meta_50000.parquet
Embeddings file: data/embeddings/multi_domain/multidomain_embeddings_50000.npy
Metadata rows: 50000
Embedding matrix: (50000, 384)
Building FAISS HNSW index...
Indexed vectors: 50000
Saved FAISS index: indices/wikipedia/multidomain_faiss_single_50k.faiss
Config written to: configs/index/config_mutidomain_faiss_50k.json
Done.


### 1b. Measure System with FAISS SINGLE Retrieval Type


In [ ]:
!python bench.py \
  --config configs/index/config_mutidomain_faiss_50k.json \
  --index multidomain_faiss_single_50k \
  --backend faiss \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-23 23:17:05.359262: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 23:17:05.378019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763939825.400388   32049 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763939825.407356   32049 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763939825.424557   32049 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## 2. FAISS SHARDED EXPERIMENT (Multidomain 50k)
FAISS-sharded runs all shard searches inside one Python process, sequentially or with local threads.

In [ ]:
!python scripts/shard_index_v2.py \
  --meta_path data/embeddings/multi_domain/multidomain_meta_50000.parquet \
  --emb_path data/embeddings/multi_domain/multidomain_embeddings_50000.npy \
  --domain multidomain \
  --out_meta_dir data/embeddings/multidomain_sharded \
  --out_vectors_dir data/embeddings/multidomain_sharded \
  --out_index_dir indices/multidomain_sharded \
  --out_manifest_dir indices/multidomain_sharded \
  --n_shards 8 \
  --save_vectors 1


Loaded 50,000 vectors (dim=384)
Normalizing vectors (L2)...
Sharding domain 'multidomain': 100% 8/8 [00:02<00:00,  3.12it/s]
Manifest written: indices/multidomain_sharded/multidomain_50000_manifest.json
Total rows = 50000


In [ ]:
!python scripts/inject_sharded_index.py \
  --config_path configs/index/config_mutidomain_faiss_50k.json \
  --index_name multidomain_faiss_sharded_50k \
  --faiss_dir indices/multidomain_sharded \
  --meta_dir data/embeddings/multidomain_sharded \
  --set_active


Successfully added FAISS-sharded entry to config json file
New index name: multidomain_faiss_sharded_50k
FAISS shards: 8
META shards: 8
Manifest: indices/multidomain_sharded/multidomain_50000_manifest.json
Updated config: configs/index/config_mutidomain_faiss_50k.json


In [ ]:
!python bench.py \
  --config configs/index/config_mutidomain_faiss_50k.json \
  --backend faiss \
  --index multidomain_faiss_sharded_50k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 \
  --iters 3 \
  --warmup 1 \
  --results_dir results \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-23 23:29:54.712731: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 23:29:54.730569: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763940594.751922   35490 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763940594.758465   35490 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763940594.774658   35490 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## 3. CHROMA EXPERIMENT (MULTIDOMAIN 50k)

In [ ]:
!python scripts/chroma_ingest_2.py \
  --meta_path data/embeddings/multi_domain/multidomain_meta_50000.parquet \
  --vec_path data/embeddings/multi_domain/multidomain_embeddings_50000.npy \
  --persist_dir indices/multidomain_chroma/multidomain_chroma_50k \
  --collection multidomain_chroma_50k

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Ingesting 50,000 records into 'multidomain_chroma_50k' @ indices/multidomain_chroma/multidomain_chroma_50k
  0% 0/50 [00:00<?, ?it/s]Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given
100% 50/50 [02:18<00:00,  2.76s/it]
✓ Manifest created: indices/multidomain_chroma/multidomain_chroma_50k/multidomain_chroma_50k_manifest.json
✓ Done.


In [ ]:
!python scripts/inject_chroma_config.py \
  --output_config configs/index/config_multidomain_chroma_50k.json \
  --index_name multidomain_chroma_50k \
  --persist_dir indices/multidomain_chroma/multidomain_chroma_50k \
  --collection_name multidomain_chroma_50k

✓ Created new Chroma config: configs/index/config_multidomain_chroma_50k.json
✓ Added index: multidomain_chroma_50k
✓ active_index = multidomain_chroma_50k


In [ ]:
!python bench.py \
--config configs/index/config_multidomain_chroma_50k.json \
--backend chroma \
--index multidomain_chroma_50k \
--qfile data/queries/all_sources_eval.txt --top_k 3 --iters 3 --warmup 1 --results_dir results/multidomain_chroma_50k \
--enable_llm \
--llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0

2025-11-23 23:57:13.611646: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 23:57:13.629831: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763942233.651470   48661 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763942233.658019   48661 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763942233.674654   48661 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## 4. SPARK EXPERIMENT (MULTIDOMAIN 50k)
Spark fan-out runs the same shard searches in parallel across Spark workers, distributing shards + merging results cluster-wide (use same .faiss and .parquet files as in FAISS Sharded Experiment)

In [ ]:
!python scripts/inject_spark_config.py \
  --index_key multidomain_spark_50000 \
  --manifest_path indices/multidomain_sharded/multidomain_50000_manifest.json \
  --out_path configs/index/config_multidomain_spark_50k.json


✓ Spark config.json created
  index_key     : multidomain_spark_50000
  manifest_path : indices/multidomain_sharded/multidomain_50000_manifest.json
  out_path      : configs/index/config_multidomain_spark_50k.json


In [ ]:
!python bench.py \
  --config configs/index/config_multidomain_spark_50k.json \
  --backend spark \
  --index multidomain_spark_50000 \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-24 00:09:45.271099: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-24 00:09:45.288989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763942985.310499   51939 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763942985.317309   51939 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763942985.334355   51939 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Experiment 2: Large LLM Without RAG vs Small LLM with RAG (Quynh)
This experiment compares a small LLM with RAG against a larger LLM without RAG to evaluate whether a 1B model paired with a datastore of size X can match the performance of a 7B model. For this comparison, the multidomain 50k datastore is used as the RAG baseline against the large standalone model.
Example:
| **Metric**                                  | **Small LLM (1.1B) + RAG (50k)** | **Big LLM (8B) No RAG** | **Interpretation**                      |
| ------------------------------------------- | -------------------------------- | ----------------------- | --------------------------------------- |
| **Retrieval Latency (ms/query)**            | ~10–30 ms                        | N/A                     | RAG adds lookup time, small cost        |
| **End-to-End Latency (ms/query)**           | ~150–300 ms                      | ~300–600 ms             | Small LLM often faster even with RAG    |
| **Inference Latency (LLM decode ms/token)** | 2–5 ms/token                     | 8–15 ms/token           | 1B model decodes ~3–5× faster           |
| **QPS (Queries Per Second)**                | Higher (1B model is fast)        | Lower                   | Big models are slower per query         |
| **Throughput (tokens/sec)**                 | High (150–350 tok/sec)           | Medium (40–120 tok/sec) | Small models decode faster              |
| **Warm-up/Load Time**                       | 0.8–1.2s                         | 2–4s                    | Big LLM takes more time to load weights |


In [ ]:
from huggingface_hub import login
login()

This command runs the evaluation using a large LLM *without* any retrieval component, directly answering each query from the evaluation file. It saves all model outputs and metrics to `results_llm_only`, allowing comparison against RAG-based systems.

The script llm_no_RAG.py measures system performance by timing how long the model takes to generate an answer for each query, then computing summary latency metrics (avg, p50, p90) from those per-query response times. More info can be found in the shared drive.

In [ ]:
!python scripts/llm_no_RAG.py \
  --model "meta-llama/Meta-Llama-3.1-8B-Instruct" \
  --qfile data/queries/all_sources_eval.txt \
  --results_dir results_llm_only


`torch_dtype` is deprecated! Use `dtype` instead!
2025-11-24 01:22:53.197126: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-24 01:22:53.216533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763947373.237381   88802 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763947373.242833   88802 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763947373.256581   88802 computation_placer.cc:177] computation placer already r

# SUMMARY:
The workflow logic for Experiments 1 and 2 is shown above. For analysis and visualization, use the bench_summary.json files located in each experiment’s output folder under ./results, as they contain the final aggregated results.

###Elita – Single-Domain Wiki 10k & 50k





In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os

# Your project folder in MyDrive
PROJECT_ROOT = "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"
os.chdir(PROJECT_ROOT)

print("Current dir:", os.getcwd())
print("Contents:", os.listdir("."))


Mounted at /content/drive
Current dir: /content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Contents: ['requirements.txt', 'Copy of Preprocessing.ipynb', 'retrieval_runner.py', 'Copy_of_Step2_Retrieval.ipynb', 'scripts', 'indices', 'data', 'configs', 'logs', 'pipeline', 'bench.py', 'results']


### Installing dependencies (for faiss, chroma, etc.)

In [ ]:
# Make sure we are in the project root
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

# Install project dependencies (includes faiss)
!pip install -r requirements.txt


/content


### Loading wiki embeddings & metadata

In [ ]:
import numpy as np
import pandas as pd

VEC_PATH = "data/embeddings/wikipedia/wiki_vectors.npy"
META_PATH = "data/embeddings/wikipedia/wiki_meta.parquet"

print("Loading full wiki vectors and metadata...")
vec = np.load(VEC_PATH)
meta = pd.read_parquet(META_PATH)

print("Full vectors shape:", vec.shape)   # e.g. (66337, 384)
print("Metadata rows:", len(meta))


Loading full wiki vectors and metadata...
Full vectors shape: (66337, 384)
Metadata rows: 66337


### Creating 10k & 50k Wiki slices

In [ ]:
import os

out_dir = "data/embeddings/wiki_single"
os.makedirs(out_dir, exist_ok=True)

sizes = [10_000, 50_000]  # ONLY 10k and 50k for your part

for n in sizes:
    if n > vec.shape[0]:
        print(f"⚠️ Skipping {n}: not enough rows in vectors ({vec.shape[0]} total).")
        continue

    v_slice = vec[:n].astype("float32")
    m_slice = meta.iloc[:n].copy()

    vec_out = f"{out_dir}/wiki_{n}.npy"
    meta_out = f"{out_dir}/wiki_{n}.parquet"

    # This will overwrite old files with the same name, which is fine
    np.save(vec_out, v_slice)
    m_slice.to_parquet(meta_out)

    print(f"✅ Saved {n} slice → {vec_out}, {meta_out}")


✅ Saved 10000 slice → data/embeddings/wiki_single/wiki_10000.npy, data/embeddings/wiki_single/wiki_10000.parquet
✅ Saved 50000 slice → data/embeddings/wiki_single/wiki_50000.npy, data/embeddings/wiki_single/wiki_50000.parquet


### Building FAISS HNSW indexes for 10k & 50k

In [ ]:
import faiss
import time
import json
import numpy as np
import os

slice_dir = "data/embeddings/wiki_single"
index_dir = "indices/wiki_single_hnsw"
os.makedirs(index_dir, exist_ok=True)

sizes = [10_000, 50_000]   # ONLY 10k & 50k
build_log = []

for n in sizes:
    vec_path = f"{slice_dir}/wiki_{n}.npy"

    if not os.path.exists(vec_path):
        print(f"⚠️ Missing {vec_path}, skipping {n}")
        continue

    xb = np.load(vec_path)
    d = xb.shape[1]
    print(f"\nBuilding HNSW index for n={n}, dim={d}...")

    index = faiss.IndexHNSWFlat(d, 32)     # HNSW with M=32
    index.hnsw.efConstruction = 200

    t0 = time.time()
    index.add(xb)
    elapsed = time.time() - t0

    out_path = f"{index_dir}/wiki_{n}_hnsw_m32.faiss"
    faiss.write_index(index, out_path)

    build_log.append({
        "n_docs": n,
        "index_path": out_path,
        "build_seconds": elapsed
    })

    print(f"✅ Built HNSW for {n} docs in {elapsed:.2f}s → {out_path}")

with open(f"{index_dir}/build_log.json", "w") as f:
    json.dump(build_log, f, indent=2)

print("\nAll done. Build log:")
print(json.dumps(build_log, indent=2))



Building HNSW index for n=10000, dim=384...
✅ Built HNSW for 10000 docs in 2.70s → indices/wiki_single_hnsw/wiki_10000_hnsw_m32.faiss

Building HNSW index for n=50000, dim=384...
✅ Built HNSW for 50000 docs in 22.55s → indices/wiki_single_hnsw/wiki_50000_hnsw_m32.faiss

All done. Build log:
[
  {
    "n_docs": 10000,
    "index_path": "indices/wiki_single_hnsw/wiki_10000_hnsw_m32.faiss",
    "build_seconds": 2.6950833797454834
  },
  {
    "n_docs": 50000,
    "index_path": "indices/wiki_single_hnsw/wiki_50000_hnsw_m32.faiss",
    "build_seconds": 22.54883098602295
  }
]


### Run bench.py for 10k

In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

!python bench.py \
  --config configs/index/config.json \
  --backend faiss \
  --index wiki_10k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 10 --iters 3 --warmup 1 \
  --results_dir results/wiki_10k


/content
2025-11-23 06:34:39.870079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763879679.890073   13672 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763879679.899346   13672 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763879679.924441   13672 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763879679.924466   13672 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763879679.924470   13672 computation_placer.cc:177] computation p

### Run bench.py for 50k

In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

!python bench.py \
  --config configs/index/config.json \
  --backend faiss \
  --index wiki_50k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 10 --iters 3 --warmup 1 \
  --results_dir results/wiki_50k


/content
2025-11-23 06:35:26.230721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763879726.250581   13971 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763879726.256654   13971 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763879726.272094   13971 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763879726.272118   13971 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763879726.272122   13971 computation_placer.cc:177] computation p

### Build FAISS shards + manifest for Wiki 10k

In [ ]:
# Install FAISS CPU version (compatible with Colab)
!pip install faiss-cpu --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 99.4 MB/s eta 0:00:00


In [ ]:
# Mount Drive and go to project root
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os

PROJECT_ROOT = "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"
os.chdir(PROJECT_ROOT)

print("Current dir:", os.getcwd())
print("Contents here:", os.listdir("."))


Mounted at /content/drive
Current dir: /content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Contents here: ['requirements.txt', 'Copy of Preprocessing.ipynb', 'retrieval_runner.py', 'Copy_of_Step2_Retrieval.ipynb', 'scripts', 'indices', 'data', 'configs', 'logs', 'pipeline', 'bench.py', 'results']


In [ ]:
# Build FAISS shards + manifest for Wiki 10k

import os
import json
import math
import numpy as np
import pandas as pd
import faiss
from pathlib import Path

# Paths for full 10k slice
VEC_10K = "data/embeddings/wiki_single/wiki_10000.npy"
META_10K = "data/embeddings/wiki_single/wiki_10000.parquet"

# Output dirs for shards
VEC_OUT_DIR_10K = Path("data/embeddings/wiki_10k_sharded")
META_OUT_DIR_10K = Path("data/embeddings/wiki_10k_sharded")
INDEX_OUT_DIR_10K = Path("indices/wiki_10k_sharded")

for p in [VEC_OUT_DIR_10K, META_OUT_DIR_10K, INDEX_OUT_DIR_10K]:
    p.mkdir(parents=True, exist_ok=True)

# Load full slice
vec_10k = np.load(VEC_10K).astype("float32")
meta_10k = pd.read_parquet(META_10K).reset_index(drop=True)

n_docs, dim = vec_10k.shape
print(f"Wiki 10k: n_docs={n_docs}, dim={dim}")

n_shards = 4  # choose 4 shards for 10k
shard_size = math.ceil(n_docs / n_shards)

manifest = {
    "domain": "wikipedia",
    "total_docs": int(n_docs),
    "dim": int(dim),
    "n_shards": int(n_shards),
    "shards": []
}

for shard_id in range(n_shards):
    start = shard_id * shard_size
    end = min((shard_id + 1) * shard_size, n_docs)
    if start >= end:
        break

    xb = vec_10k[start:end]
    m = meta_10k.iloc[start:end].copy()

    print(f"  Shard {shard_id}: docs [{start}:{end}) → {len(m)} docs")

    # Build HNSW index for shard
    index = faiss.IndexHNSWFlat(dim, 32)
    index.hnsw.efConstruction = 200
    index.add(xb)

    index_path = INDEX_OUT_DIR_10K / f"wiki_10000_shard_{shard_id:02d}.faiss"
    meta_path = META_OUT_DIR_10K / f"wiki_10000_meta_shard_{shard_id:02d}.parquet"

    faiss.write_index(index, str(index_path))
    m.to_parquet(meta_path)

    manifest["shards"].append({
        "shard_id": shard_id,
        "start": int(start),
        "end": int(end),
        "index_path": str(index_path),
        "meta_path": str(meta_path),
        "n_docs": int(len(m)),
    })

# Save manifest
manifest_path_10k = INDEX_OUT_DIR_10K / "wiki_10000_manifest.json"
with open(manifest_path_10k, "w") as f:
    json.dump(manifest, f, indent=2)

print(f"\n✅ Wiki 10k sharded into {len(manifest['shards'])} shards.")
print(f"Manifest saved to: {manifest_path_10k}")


Wiki 10k: n_docs=10000, dim=384
  Shard 0: docs [0:2500) → 2500 docs
  Shard 1: docs [2500:5000) → 2500 docs
  Shard 2: docs [5000:7500) → 2500 docs
  Shard 3: docs [7500:10000) → 2500 docs

✅ Wiki 10k sharded into 4 shards.
Manifest saved to: indices/wiki_10k_sharded/wiki_10000_manifest.json


### Build FAISS shards + manifest for Wiki 50k

In [ ]:
# Build FAISS shards + manifest for Wiki 50k

import os
import json
import math
import numpy as np
import pandas as pd
import faiss
from pathlib import Path

# Paths for full 50k slice
VEC_50K = "data/embeddings/wiki_single/wiki_50000.npy"
META_50K = "data/embeddings/wiki_single/wiki_50000.parquet"

# Output dirs for shards
VEC_OUT_DIR_50K = Path("data/embeddings/wiki_50k_sharded")
META_OUT_DIR_50K = Path("data/embeddings/wiki_50k_sharded")
INDEX_OUT_DIR_50K = Path("indices/wiki_50k_sharded")

for p in [VEC_OUT_DIR_50K, META_OUT_DIR_50K, INDEX_OUT_DIR_50K]:
    p.mkdir(parents=True, exist_ok=True)

# Load full slice
vec_50k = np.load(VEC_50K).astype("float32")
meta_50k = pd.read_parquet(META_50K).reset_index(drop=True)

n_docs, dim = vec_50k.shape
print(f"Wiki 50k: n_docs={n_docs}, dim={dim}")

n_shards = 8  # choose 8 shards for 50k
shard_size = math.ceil(n_docs / n_shards)

manifest_50k = {
    "domain": "wikipedia",
    "total_docs": int(n_docs),
    "dim": int(dim),
    "n_shards": int(n_shards),
    "shards": []
}

for shard_id in range(n_shards):
    start = shard_id * shard_size
    end = min((shard_id + 1) * shard_size, n_docs)
    if start >= end:
        break

    xb = vec_50k[start:end]
    m = meta_50k.iloc[start:end].copy()

    print(f"  Shard {shard_id}: docs [{start}:{end}) → {len(m)} docs")

    # Build HNSW index for shard
    index = faiss.IndexHNSWFlat(dim, 32)
    index.hnsw.efConstruction = 200
    index.add(xb)

    index_path = INDEX_OUT_DIR_50K / f"wiki_50000_shard_{shard_id:02d}.faiss"
    meta_path = META_OUT_DIR_50K / f"wiki_50000_meta_shard_{shard_id:02d}.parquet"

    faiss.write_index(index, str(index_path))
    m.to_parquet(meta_path)

    manifest_50k["shards"].append({
        "shard_id": shard_id,
        "start": int(start),
        "end": int(end),
        "index_path": str(index_path),
        "meta_path": str(meta_path),
        "n_docs": int(len(m)),
    })

# Save manifest
manifest_path_50k = INDEX_OUT_DIR_50K / "wiki_50000_manifest.json"
with open(manifest_path_50k, "w") as f:
    json.dump(manifest_50k, f, indent=2)

print(f"\n✅ Wiki 50k sharded into {len(manifest_50k['shards'])} shards.")
print(f"Manifest saved to: {manifest_path_50k}")


Wiki 50k: n_docs=50000, dim=384
  Shard 0: docs [0:6250) → 6250 docs
  Shard 1: docs [6250:12500) → 6250 docs
  Shard 2: docs [12500:18750) → 6250 docs
  Shard 3: docs [18750:25000) → 6250 docs
  Shard 4: docs [25000:31250) → 6250 docs
  Shard 5: docs [31250:37500) → 6250 docs
  Shard 6: docs [37500:43750) → 6250 docs
  Shard 7: docs [43750:50000) → 6250 docs

✅ Wiki 50k sharded into 8 shards.
Manifest saved to: indices/wiki_50k_sharded/wiki_50000_manifest.json


### Build Chroma collection for Wiki 10k

In [ ]:
!pip install chromadb --no-cache-dir


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 383.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 405.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 380.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 146.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 340.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 395.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 214.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 226.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 375.6 

In [ ]:
# Mount Drive and go to project root
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os

PROJECT_ROOT = "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"
os.chdir(PROJECT_ROOT)

print("Current dir:", os.getcwd())
print("Contents here:", os.listdir("."))

Mounted at /content/drive
Current dir: /content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Contents here: ['requirements.txt', 'Copy of Preprocessing.ipynb', 'retrieval_runner.py', 'Copy_of_Step2_Retrieval.ipynb', 'scripts', 'indices', 'data', 'configs', 'logs', 'pipeline', 'bench.py', 'results']


In [ ]:
# Build Chroma collection for Wiki 10k

import os
import numpy as np
import pandas as pd
import chromadb
from chromadb.config import Settings
from pathlib import Path

VEC_10K = "data/embeddings/wiki_single/wiki_10000.npy"
META_10K = "data/embeddings/wiki_single/wiki_10000.parquet"

CHROMA_DIR_10K = Path("indices/wiki_chroma/wiki_chroma_10k")
CHROMA_DIR_10K.mkdir(parents=True, exist_ok=True)

# Load data
vec_10k = np.load(VEC_10K).astype("float32")
meta_10k = pd.read_parquet(META_10K).reset_index(drop=True)

print("10k vectors shape:", vec_10k.shape)
print("10k meta rows:", len(meta_10k))

# Start Chroma client
client_10k = chromadb.PersistentClient(
    path=str(CHROMA_DIR_10K),
    settings=Settings(allow_reset=True)
)

collection_name_10k = "wiki_chroma_10k"

# If exists, drop and recreate
for coll in client_10k.list_collections():
    if coll.name == collection_name_10k:
        client_10k.delete_collection(collection_name_10k)
        break

coll_10k = client_10k.create_collection(name=collection_name_10k)

# Prepare IDs, texts, metadatas
ids = [f"wiki10k_{i}" for i in range(len(meta_10k))]
texts = meta_10k["text"].astype(str).tolist() if "text" in meta_10k.columns else [""]*len(meta_10k)
metadatas = meta_10k.to_dict(orient="records")

# Upsert in batches
batch_size = 1000
for start in range(0, len(ids), batch_size):
    end = start + batch_size
    coll_10k.add(
        ids=ids[start:end],
        documents=texts[start:end],
        metadatas=metadatas[start:end],
        embeddings=vec_10k[start:end].tolist()
    )
    print(f"  Upserted {end} / {len(ids)}")

print(f"\n✅ Built Chroma collection '{collection_name_10k}' at {CHROMA_DIR_10K}")


10k vectors shape: (10000, 384)
10k meta rows: 10000
  Upserted 1000 / 10000
  Upserted 2000 / 10000
  Upserted 3000 / 10000
  Upserted 4000 / 10000
  Upserted 5000 / 10000
  Upserted 6000 / 10000
  Upserted 7000 / 10000
  Upserted 8000 / 10000
  Upserted 9000 / 10000
  Upserted 10000 / 10000

✅ Built Chroma collection 'wiki_chroma_10k' at indices/wiki_chroma/wiki_chroma_10k


### Build Chroma collection for Wiki 50k

In [ ]:
# Build Chroma collection for Wiki 50k

import os
import numpy as np
import pandas as pd
import chromadb
from chromadb.config import Settings
from pathlib import Path

VEC_50K = "data/embeddings/wiki_single/wiki_50000.npy"
META_50K = "data/embeddings/wiki_single/wiki_50000.parquet"

CHROMA_DIR_50K = Path("indices/wiki_chroma/wiki_chroma_50k")
CHROMA_DIR_50K.mkdir(parents=True, exist_ok=True)

# Load data
vec_50k = np.load(VEC_50K).astype("float32")
meta_50k = pd.read_parquet(META_50K).reset_index(drop=True)

print("50k vectors shape:", vec_50k.shape)
print("50k meta rows:", len(meta_50k))

# Start Chroma client
client_50k = chromadb.PersistentClient(
    path=str(CHROMA_DIR_50K),
    settings=Settings(allow_reset=True)
)

collection_name_50k = "wiki_chroma_50k"

# If exists, drop and recreate
for coll in client_50k.list_collections():
    if coll.name == collection_name_50k:
        client_50k.delete_collection(collection_name_50k)
        break

coll_50k = client_50k.create_collection(name=collection_name_50k)

# Prepare IDs, texts, metadatas
ids = [f"wiki50k_{i}" for i in range(len(meta_50k))]
texts = meta_50k["text"].astype(str).tolist() if "text" in meta_50k.columns else [""]*len(meta_50k)
metadatas = meta_50k.to_dict(orient="records")

# Upsert in batches
batch_size = 1000
for start in range(0, len(ids), batch_size):
    end = start + batch_size
    coll_50k.add(
        ids=ids[start:end],
        documents=texts[start:end],
        metadatas=metadatas[start:end],
        embeddings=vec_50k[start:end].tolist()
    )
    print(f"  Upserted {end} / {len(ids)}")

print(f"\n✅ Built Chroma collection '{collection_name_50k}' at {CHROMA_DIR_50K}")


50k vectors shape: (50000, 384)
50k meta rows: 50000
  Upserted 1000 / 50000
  Upserted 2000 / 50000
  Upserted 3000 / 50000
  Upserted 4000 / 50000
  Upserted 5000 / 50000
  Upserted 6000 / 50000
  Upserted 7000 / 50000
  Upserted 8000 / 50000
  Upserted 9000 / 50000
  Upserted 10000 / 50000
  Upserted 11000 / 50000
  Upserted 12000 / 50000
  Upserted 13000 / 50000
  Upserted 14000 / 50000
  Upserted 15000 / 50000
  Upserted 16000 / 50000
  Upserted 17000 / 50000
  Upserted 18000 / 50000
  Upserted 19000 / 50000
  Upserted 20000 / 50000
  Upserted 21000 / 50000
  Upserted 22000 / 50000
  Upserted 23000 / 50000
  Upserted 24000 / 50000
  Upserted 25000 / 50000
  Upserted 26000 / 50000
  Upserted 27000 / 50000
  Upserted 28000 / 50000
  Upserted 29000 / 50000
  Upserted 30000 / 50000
  Upserted 31000 / 50000
  Upserted 32000 / 50000
  Upserted 33000 / 50000
  Upserted 34000 / 50000
  Upserted 35000 / 50000
  Upserted 36000 / 50000
  Upserted 37000 / 50000
  Upserted 38000 / 50000
  Upse

### Checking for spark files:

In [ ]:
# Check for spark backend files
!ls pipeline/backends


backup	 chroma_backend.py  __pycache__
base.py  faiss_backend.py   spark_backend.py


In [ ]:
# Check scripts folder for spark fanout
!ls scripts | grep -i spark


inject_spark_config.py
spark_fanout.py


In [ ]:
!ls configs/index


backup				    config_tmp.json
config.json			    config_wiki_chroma_30k.json
config_multidomain_chroma_50k.json  config_wiki_spark_30k.json
config_multidomain_spark_50k.json   wiki_30k_config.json
config_mutidomain_faiss_50k.json


### Create Spark config for Wiki 10k

In [ ]:
# Create Spark config for Wiki 10k (uses wiki_10000_manifest.json)

!python scripts/inject_spark_config.py \
  --index_key wiki_spark_10000 \
  --manifest_path indices/wiki_10k_sharded/wiki_10000_manifest.json \
  --out_path configs/index/config_wiki_spark_10k.json

✓ Spark config.json created
  index_key     : wiki_spark_10000
  manifest_path : indices/wiki_10k_sharded/wiki_10000_manifest.json
  out_path      : configs/index/config_wiki_spark_10k.json


### Run Spark benchmark for Wiki 10k

In [ ]:
!python bench.py \
  --config configs/index/config_wiki_spark_10k.json \
  --backend spark \
  --index wiki_spark_10000 \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 \
  --iters 3 \
  --warmup 1 \
  --results_dir results/wiki_spark_10k \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-27 04:31:48.811037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764217908.830977   17806 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764217908.837091   17806 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764217908.853686   17806 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764217908.853712   17806 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764217908.853716   17806 computation_placer.cc:177] computation placer alr

### Create Spark config for Wiki 50k

In [ ]:
# Create Spark config for Wiki 50k (uses wiki_50000_manifest.json)

!python scripts/inject_spark_config.py \
  --index_key wiki_spark_50000 \
  --manifest_path indices/wiki_50k_sharded/wiki_50000_manifest.json \
  --out_path configs/index/config_wiki_spark_50k.json


✓ Spark config.json created
  index_key     : wiki_spark_50000
  manifest_path : indices/wiki_50k_sharded/wiki_50000_manifest.json
  out_path      : configs/index/config_wiki_spark_50k.json


### Run Spark benchmark for Wiki 50k

In [ ]:
# Run bench.py with backend=SPARK for Wiki 50k

!python bench.py \
  --config configs/index/config_wiki_spark_50k.json \
  --backend spark \
  --index wiki_spark_50000 \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 \
  --iters 3 \
  --warmup 1 \
  --results_dir results/wiki_spark_50k \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-27 05:30:38.329504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764221438.353147   37329 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764221438.360081   37329 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764221438.378153   37329 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764221438.378189   37329 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764221438.378196   37329 computation_placer.cc:177] computation placer alr

### Organizing the outputs to results folder.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os

PROJECT_ROOT = "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"
os.chdir(PROJECT_ROOT)

print("Current dir:", os.getcwd())
print("Results dir contents:", os.listdir("results"))


Mounted at /content/drive
Current dir: /content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Results dir contents: ['spark_fanout_hits.csv', 'multi_default', 'multi_sharded', 'multi_chroma', 'wiki_faiss_single_30k', 'wiki_faiss_sharded_30k', 'wiki_chroma_30k', 'wiki_spark_30000', 'wiki_10k', 'wiki_50k', 'multidomain_faiss_single_50k', 'multidomain_faiss_sharded_50k', 'multidomain_chroma_50k', 'multidomain_spark_50000', 'results_llm_only', 'wiki_spark_10k', 'wiki_spark_50k']


### Helper to reorganize result folders (no rerun)

In [ ]:
# 🧹 Helper to reorganize existing benchmark results by backend

import os
import json
import glob
import shutil

RESULTS_ROOT = os.path.join(PROJECT_ROOT, "results")

def reorganize_wiki_size(size: str):
    """
    size: "10k" or "50k"
    It looks inside results/wiki_<size>/wiki_<size>/* and
    moves each timestamp folder into:
      - results/wiki_faiss_single_<size>/...
      - results/wiki_chroma_<size>/...
    based on bench_summary.json["backend"].
    """
    src_root = os.path.join(RESULTS_ROOT, f"wiki_{size}", f"wiki_{size}")
    print(f"Scanning: {src_root}")

    if not os.path.isdir(src_root):
        print("  ⚠ Source dir not found, skipping.")
        return

    run_dirs = sorted(glob.glob(os.path.join(src_root, "*/")))
    if not run_dirs:
        print("  (No timestamp folders found, nothing to move.)")
        return

    for run_dir in run_dirs:
        summary_path = os.path.join(run_dir, "bench_summary.json")
        if not os.path.exists(summary_path):
            print(f"  ⚠ No bench_summary.json in {run_dir}, skipping.")
            continue

        with open(summary_path, "r") as f:
            js = json.load(f)

        backend = js.get("backend", "unknown")
        index_key = js.get("index_key", "unknown")

        # Decide destination root based on backend
        if backend == "faiss":
            dest_root = os.path.join(RESULTS_ROOT, f"wiki_faiss_single_{size}")
        elif backend == "chroma":
            dest_root = os.path.join(RESULTS_ROOT, f"wiki_chroma_{size}")
        elif backend == "spark":
            # In practice we didn't store spark runs here, but just in case:
            dest_root = os.path.join(RESULTS_ROOT, f"wiki_spark_{size}")
        else:
            dest_root = os.path.join(RESULTS_ROOT, f"wiki_other_{size}")

        os.makedirs(dest_root, exist_ok=True)

        dest_dir = os.path.join(dest_root, os.path.basename(run_dir.rstrip("/")))

        print(f"  Moving {run_dir} → {dest_dir}")
        shutil.move(run_dir, dest_dir)

        print(f"    backend={backend}, index_key={index_key}")

    print(f"✅ Done reorganizing wiki_{size}.")


### Reorganize Wiki 10k

In [ ]:
reorganize_wiki_size("10k")


Scanning: /content/drive/MyDrive/AMS 560 PROJECT_SCALING/results/wiki_10k/wiki_10k
  Moving /content/drive/MyDrive/AMS 560 PROJECT_SCALING/results/wiki_10k/wiki_10k/20251123-063455-2471ccb3ef24/ → /content/drive/MyDrive/AMS 560 PROJECT_SCALING/results/wiki_faiss_single_10k/20251123-063455-2471ccb3ef24
    backend=faiss, index_key=wiki_10k
  Moving /content/drive/MyDrive/AMS 560 PROJECT_SCALING/results/wiki_10k/wiki_10k/20251123-234731-2471ccb3ef24/ → /content/drive/MyDrive/AMS 560 PROJECT_SCALING/results/wiki_faiss_single_10k/20251123-234731-2471ccb3ef24
    backend=faiss, index_key=wiki_10k
✅ Done reorganizing wiki_10k.


### Reorganize Wiki 50k

In [ ]:
reorganize_wiki_size("50k")


Scanning: /content/drive/MyDrive/AMS 560 PROJECT_SCALING/results/wiki_50k/wiki_50k
  Moving /content/drive/MyDrive/AMS 560 PROJECT_SCALING/results/wiki_50k/wiki_50k/20251123-063532-59624110e331/ → /content/drive/MyDrive/AMS 560 PROJECT_SCALING/results/wiki_faiss_single_50k/20251123-063532-59624110e331
    backend=faiss, index_key=wiki_50k
  Moving /content/drive/MyDrive/AMS 560 PROJECT_SCALING/results/wiki_50k/wiki_50k/20251124-002202-59624110e331/ → /content/drive/MyDrive/AMS 560 PROJECT_SCALING/results/wiki_faiss_single_50k/20251124-002202-59624110e331
    backend=faiss, index_key=wiki_50k
✅ Done reorganizing wiki_50k.


### FRESH START FOR SCALING PART BY ELITA

### Setup:

In [ ]:
# === Cell 0: Mount Drive & go to project root ===
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

import os

PROJECT_ROOT = "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"
os.chdir(PROJECT_ROOT)

print("Current dir:", os.getcwd())
print("Contents here:", os.listdir("."))


Mounted at /content/drive/
Current dir: /content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Contents here: ['requirements.txt', 'Copy of Preprocessing.ipynb', 'retrieval_runner.py', 'Copy_of_Step2_Retrieval.ipynb', 'scripts', 'indices', 'data', 'configs', 'logs', 'pipeline', 'bench.py', 'results']


### Install deps

In [ ]:
# === Cell 1: Install dependencies (once per runtime) ===
!pip install -r requirements.txt
!apt-get install -y tree
!pip install radon

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 0 B/47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Selecting previously unselected package tree.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.9 MB/s eta 0:00:00


### Imports

In [ ]:
# === Cell 2: Core imports for scaling / backends / Spark ===
import os, glob, json
import numpy as np
import pandas as pd
import pyarrow
import chromadb
import faiss
import pyspark

from sentence_transformers import SentenceTransformer

print("Imports OK.")


Imports OK.


### Quick sanity check

In [ ]:
# === Cell 3: Sanity check folders (indices, configs, results) ===

def ls(path):
    print(f"\n=== {path} ===")
    try:
        for x in sorted(os.listdir(path)):
            print(" ", x)
    except FileNotFoundError:
        print("  (missing)")

ls("indices")
ls("indices/wikipedia")
ls("indices/wiki_sharded")
ls("indices/wiki_chroma")
ls("configs/index")
ls("results")



=== indices ===
  chroma_store
  multi_default
  multi_default_chroma_manifest.json
  multi_default_sharded
  multidomain_chroma
  multidomain_sharded
  pubmed
  sample_hnsw.faiss
  stackexchange
  wiki_10k_sharded
  wiki_50k_sharded
  wiki_chroma
  wiki_sharded
  wiki_single_hnsw
  wikipedia

=== indices/wikipedia ===
  multidomain_faiss_single_30k.faiss
  multidomain_faiss_single_50k.faiss
  wiki_faiss_single_30k.faiss
  wiki_hnsw.faiss

=== indices/wiki_sharded ===
  wiki_30000_manifest.json
  wiki_30000_shard_000.faiss
  wiki_30000_shard_001.faiss
  wiki_30000_shard_002.faiss
  wiki_30000_shard_003.faiss
  wiki_30000_shard_004.faiss
  wiki_30000_shard_005.faiss
  wiki_30000_shard_006.faiss
  wiki_30000_shard_007.faiss

=== indices/wiki_chroma ===
  wiki_chroma_10k
  wiki_chroma_30k
  wiki_chroma_50k

=== configs/index ===
  backup
  config.json
  config_multidomain_chroma_50k.json
  config_multidomain_spark_50k.json
  config_mutidomain_faiss_50k.json
  config_tmp.json
  config_wik

### Rebuild Wiki 10k slice

In [ ]:
# === WIKI 10k: Rebuild 10k slice ===

!python scripts/embedding_slice.py \
  --emb_path data/embeddings/wikipedia/wiki_vectors.npy \
  --meta_path data/embeddings/wikipedia/wiki_meta.parquet \
  --out_dir data/embeddings/wikipedia \
  --n 10000 \
  --domain wiki


Loading metadata: data/embeddings/wikipedia/wiki_meta.parquet
Metadata rows available: 66337
Slicing first 10000 metadata rows...
Saved: data/embeddings/wikipedia/wiki_meta_10000.parquet
Loading embeddings: data/embeddings/wikipedia/wiki_vectors.npy
Embedding matrix shape: (66337, 384)
Slicing first 10000 embedding rows...
Saved: data/embeddings/wikipedia/wiki_embeddings_10000.npy
Files created:
  - data/embeddings/wikipedia/wiki_meta_10000.parquet
  - data/embeddings/wikipedia/wiki_embeddings_10000.npy


### Rebuild Wiki 50k slice

In [ ]:
# === WIKI 50k: Rebuild 50k slice ===

!python scripts/embedding_slice.py \
  --emb_path data/embeddings/wikipedia/wiki_vectors.npy \
  --meta_path data/embeddings/wikipedia/wiki_meta.parquet \
  --out_dir data/embeddings/wikipedia \
  --n 50000 \
  --domain wiki


Loading metadata: data/embeddings/wikipedia/wiki_meta.parquet
Metadata rows available: 66337
Slicing first 50000 metadata rows...
Saved: data/embeddings/wikipedia/wiki_meta_50000.parquet
Loading embeddings: data/embeddings/wikipedia/wiki_vectors.npy
Embedding matrix shape: (66337, 384)
Slicing first 50000 embedding rows...
Saved: data/embeddings/wikipedia/wiki_embeddings_50000.npy
Files created:
  - data/embeddings/wikipedia/wiki_meta_50000.parquet
  - data/embeddings/wikipedia/wiki_embeddings_50000.npy


### To check if files exist:

In [ ]:
import os

for f in sorted(os.listdir("data/embeddings/wikipedia")):
    if "10000" in f or "50000" in f or "30000" in f:
        print(f)


wiki_embeddings_10000.npy
wiki_embeddings_30000.npy
wiki_embeddings_50000.npy
wiki_meta_10000.parquet
wiki_meta_30000.parquet
wiki_meta_50000.parquet


### FAISS SINGLE – WIKI 10k

In [ ]:
# === FIX: Rebuild correct 10k FAISS index from 10k embeddings ===

import numpy as np
import faiss
import os

emb_path = "data/embeddings/wikipedia/wiki_embeddings_10000.npy"
out_path = "indices/wikipedia/wiki_faiss_single_10k.faiss"

# Load 10k embedding matrix
vecs = np.load(emb_path).astype("float32")
print("Loaded embeddings:", vecs.shape)  # should be (10000, 384)

n, d = vecs.shape

# Normalize (safe even if already normalized)
faiss.normalize_L2(vecs)

# Build HNSW index (M=32 like other HNSW indexes in this project)
index = faiss.IndexHNSWFlat(d, 32)
index.hnsw.efConstruction = 200

print("Building 10k HNSW index...")
index.add(vecs)
print("Indexed vectors:", index.ntotal)

# Overwrite wrong 10k index with correct one
faiss.write_index(index, out_path)
print("✅ Saved corrected 10k FAISS index to:", out_path)


Loaded embeddings: (10000, 384)
Building 10k HNSW index...
Indexed vectors: 10000
✅ Saved corrected 10k FAISS index to: indices/wikipedia/wiki_faiss_single_10k.faiss


In [ ]:
# === FIX: Point wiki_10k config to 10k meta file ===
import json

conf_path = "configs/index/wiki_10k_config.json"

with open(conf_path, "r") as f:
    cfg = json.load(f)

print("Before fix:\n", cfg, "\n")

# Try to find the index block
if "indices" in cfg:
    # Case 1: index explicitly named
    key = "wiki_faiss_single_10k"
    if key in cfg["indices"]:
        cfg["indices"][key]["faiss_path"] = "indices/wikipedia/wiki_faiss_single_10k.faiss"
        cfg["indices"][key]["meta_path"] = "data/embeddings/wikipedia/wiki_meta_10000.parquet"
    else:
        # Case 2: use active_index as fallback
        active = cfg.get("active_index")
        if active and active in cfg["indices"]:
            cfg["indices"][active]["faiss_path"] = "indices/wikipedia/wiki_faiss_single_10k.faiss"
            cfg["indices"][active]["meta_path"] = "data/embeddings/wikipedia/wiki_meta_10000.parquet"

with open(conf_path, "w") as f:
    json.dump(cfg, f, indent=2)

print("\n✅ After fix (faiss_path + meta_path patched):\n", cfg)


Before fix:
 {'indices': {'wiki_faiss_single_10k': {'backend': 'faiss', 'faiss_path': 'indices/wikipedia/wiki_faiss_single_10k.faiss', 'meta_path': 'data/embeddings/wikipedia/wiki_meta_50000.parquet', 'model_name': 'BAAI/bge-small-en-v1.5', 'metric': 'ip', 'normalize_query': True, 'index_metric': 'ip', 'vector_normed': True, 'skip_path_checks': False}}} 


✅ After fix (faiss_path + meta_path patched):
 {'indices': {'wiki_faiss_single_10k': {'backend': 'faiss', 'faiss_path': 'indices/wikipedia/wiki_faiss_single_10k.faiss', 'meta_path': 'data/embeddings/wikipedia/wiki_meta_10000.parquet', 'model_name': 'BAAI/bge-small-en-v1.5', 'metric': 'ip', 'normalize_query': True, 'index_metric': 'ip', 'vector_normed': True, 'skip_path_checks': False}}}


In [ ]:
# === WIKI 10k: FAISS single benchmark ===
!python bench.py \
  --config configs/index/wiki_10k_config.json \
  --backend faiss \
  --index wiki_faiss_single_10k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 10 \
  --iters 3 \
  --warmup 1 \
  --results_dir results/wiki_faiss_single_10k



2025-11-29 02:12:22.236475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764382342.257866    2354 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764382342.263757    2354 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764382342.278951    2354 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764382342.278976    2354 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764382342.278980    2354 computation_placer.cc:177] computation placer alr

### FAISS SINGLE – WIKI 50k

In [ ]:
!python scripts/build_faiss_index.py \
  --subset_dir data/embeddings/wikipedia \
  --index_group wikipedia \
  --index_name wiki_faiss_single_50k \
  --domain wiki \
  --out_config configs/index/wiki_50k_config.json


Metadata file: data/embeddings/wikipedia/wiki_meta_50000.parquet
Embeddings file: data/embeddings/wikipedia/wiki_embeddings_50000.npy
Metadata rows: 50000
Embedding matrix: (50000, 384)
Building FAISS HNSW index...
Indexed vectors: 50000
Saved FAISS index: indices/wikipedia/wiki_faiss_single_50k.faiss
Config written to: configs/index/wiki_50k_config.json
Done.


In [ ]:
# === WIKI 50k: FAISS single benchmark ===
!python bench.py \
  --config configs/index/wiki_50k_config.json \
  --backend faiss \
  --index wiki_faiss_single_50k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 10 \
  --iters 3 \
  --warmup 1 \
  --results_dir results/wiki_faiss_single_50k


2025-11-29 02:13:19.234026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764382399.254048    2698 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764382399.260103    2698 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764382399.275233    2698 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764382399.275259    2698 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764382399.275263    2698 computation_placer.cc:177] computation placer alr

In [ ]:
!pip install -r requirements.txt


In [ ]:
!pip install chromadb
!pip install faiss-cpu
!pip install pyspark


In [ ]:
!ls indices/wikipedia
!ls indices/wiki_10k_sharded
!ls indices/wiki_50k_sharded
!ls indices/wiki_chroma
!ls configs/index


multidomain_faiss_single_30k.faiss  wiki_faiss_single_30k.faiss
multidomain_faiss_single_50k.faiss  wiki_faiss_single_50k.faiss
wiki_faiss_single_10k.faiss	    wiki_hnsw.faiss
wiki_10000_manifest.json   wiki_10000_shard_01.faiss  wiki_10000_shard_03.faiss
wiki_10000_shard_00.faiss  wiki_10000_shard_02.faiss
wiki_50000_manifest.json   wiki_50000_shard_02.faiss  wiki_50000_shard_05.faiss
wiki_50000_shard_00.faiss  wiki_50000_shard_03.faiss  wiki_50000_shard_06.faiss
wiki_50000_shard_01.faiss  wiki_50000_shard_04.faiss  wiki_50000_shard_07.faiss
wiki_chroma_10k  wiki_chroma_30k  wiki_chroma_50k
backup				    config_wiki_spark_10k.json
config.json			    config_wiki_spark_30k.json
config_multidomain_chroma_50k.json  config_wiki_spark_50k.json
config_multidomain_spark_50k.json   wiki_10k_config.json
config_mutidomain_faiss_50k.json    wiki_30k_config.json
config_tmp.json			    wiki_50k_config.json
config_wiki_chroma_30k.json


### Wiki 10k – Faiss Single + LLM (TinyLlama)

In [ ]:
!python bench.py \
  --config configs/index/wiki_10k_config.json \
  --index wiki_faiss_single_10k \
  --backend faiss \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/wiki_faiss_single_10k_llm \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-29 02:49:02.997825: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764384543.018395   11959 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764384543.024695   11959 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764384543.041169   11959 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764384543.041198   11959 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764384543.041202   11959 computation_placer.cc:177] computation placer alr

### Wiki 50k – Faiss Single + LLM (TinyLlama)

In [ ]:
!python bench.py \
  --config configs/index/wiki_50k_config.json \
  --index wiki_faiss_single_50k \
  --backend faiss \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/wiki_faiss_single_50k_llm \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-29 03:21:14.907700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764386474.927828   20276 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764386474.934021   20276 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764386474.949227   20276 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764386474.949260   20276 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764386474.949264   20276 computation_placer.cc:177] computation placer alr

### Quick sanity check!

In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

!ls data/embeddings/wikipedia | grep wiki_meta_
!ls data/embeddings/wikipedia | grep wiki_embeddings_


/content
wiki_meta_10000.parquet
wiki_meta_30000.parquet
wiki_meta_50000.parquet
wiki_embeddings_10000.npy
wiki_embeddings_30000.npy
wiki_embeddings_50000.npy


### FAISS SHARDED – Wiki 10k & 50k

#### Shard Wiki 10k

In [ ]:
!python scripts/shard_index_v2.py \
  --meta_path data/embeddings/wikipedia/wiki_meta_10000.parquet \
  --emb_path data/embeddings/wikipedia/wiki_embeddings_10000.npy \
  --domain wiki \
  --out_meta_dir data/embeddings/wiki_sharded_10k \
  --out_vectors_dir data/embeddings/wiki_sharded_10k \
  --out_index_dir indices/wiki_sharded_10k \
  --out_manifest_dir indices/wiki_sharded_10k \
  --n_shards 4 \
  --save_vectors 1


Loaded 10,000 vectors (dim=384)
Normalizing vectors (L2)...
Sharding domain 'wiki': 100% 4/4 [00:01<00:00,  2.41it/s]
Manifest written: indices/wiki_sharded_10k/wiki_10000_manifest.json
Total rows = 10000


#### Inject 10k sharded index into config

In [ ]:
!python scripts/inject_sharded_index.py \
  --config_path configs/index/wiki_10k_config.json \
  --index_name wiki_faiss_sharded_10k \
  --faiss_dir indices/wiki_sharded_10k \
  --meta_dir data/embeddings/wiki_sharded_10k \
  --set_active



Successfully added FAISS-sharded entry to config json file
New index name: wiki_faiss_sharded_10k
FAISS shards: 4
META shards: 4
Manifest: indices/wiki_sharded_10k/wiki_10000_manifest.json
Updated config: configs/index/wiki_10k_config.json


#### Benchmark Wiki 10k FAISS sharded + LLM

In [ ]:
!python bench.py \
  --config configs/index/wiki_10k_config.json \
  --backend faiss \
  --index wiki_faiss_sharded_10k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 \
  --iters 3 \
  --warmup 1 \
  --results_dir results/wiki_faiss_sharded_10k_llm \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-29 04:12:57.581057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764389577.601347   33510 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764389577.607570   33510 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764389577.623525   33510 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764389577.623553   33510 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764389577.623557   33510 computation_placer.cc:177] computation placer alr

#### Shard Wiki 50k

In [ ]:
!python scripts/shard_index_v2.py \
  --meta_path data/embeddings/wikipedia/wiki_meta_50000.parquet \
  --emb_path data/embeddings/wikipedia/wiki_embeddings_50000.npy \
  --domain wiki \
  --out_meta_dir data/embeddings/wiki_sharded_50k \
  --out_vectors_dir data/embeddings/wiki_sharded_50k \
  --out_index_dir indices/wiki_sharded_50k \
  --out_manifest_dir indices/wiki_sharded_50k \
  --n_shards 8 \
  --save_vectors 1


Loaded 50,000 vectors (dim=384)
Normalizing vectors (L2)...
Sharding domain 'wiki': 100% 8/8 [00:13<00:00,  1.68s/it]
Manifest written: indices/wiki_sharded_50k/wiki_50000_manifest.json
Total rows = 50000


#### Inject 50k sharded index into config

In [ ]:
!python scripts/inject_sharded_index.py \
  --config_path configs/index/wiki_50k_config.json \
  --index_name wiki_faiss_sharded_50k \
  --faiss_dir indices/wiki_sharded_50k \
  --meta_dir data/embeddings/wiki_sharded_50k \
  --set_active


Successfully added FAISS-sharded entry to config json file
New index name: wiki_faiss_sharded_50k
FAISS shards: 8
META shards: 8
Manifest: indices/wiki_sharded_50k/wiki_50000_manifest.json
Updated config: configs/index/wiki_50k_config.json


#### Benchmark Wiki 50k FAISS sharded + LLM

In [ ]:
!python bench.py \
  --config configs/index/wiki_50k_config.json \
  --backend faiss \
  --index wiki_faiss_sharded_50k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 \
  --iters 3 \
  --warmup 1 \
  --results_dir results/wiki_faiss_sharded_50k_llm \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-29 04:43:01.027368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764391381.059933   41148 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764391381.070065   41148 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764391381.096135   41148 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764391381.096177   41148 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764391381.096185   41148 computation_placer.cc:177] computation placer alr

### Chroma for Wiki 10k & 50k

#### Create Chroma configs for 10k and 50k

In [ ]:
!python scripts/inject_chroma_config.py \
  --output_config configs/index/config_wiki_chroma_10k.json \
  --index_name wiki_chroma_10k \
  --persist_dir indices/wiki_chroma/wiki_chroma_10k \
  --collection_name wiki_chroma_10k


✓ Created new Chroma config: configs/index/config_wiki_chroma_10k.json
✓ Added index: wiki_chroma_10k
✓ active_index = wiki_chroma_10k


#### config for Wiki Chroma 50k

In [ ]:
!python scripts/inject_chroma_config.py \
  --output_config configs/index/config_wiki_chroma_50k.json \
  --index_name wiki_chroma_50k \
  --persist_dir indices/wiki_chroma/wiki_chroma_50k \
  --collection_name wiki_chroma_50k


✓ Created new Chroma config: configs/index/config_wiki_chroma_50k.json
✓ Added index: wiki_chroma_50k
✓ active_index = wiki_chroma_50k


#### Benchmark Chroma 10k & 50k (with LLM)

#### Wiki Chroma 10k + LLM

In [ ]:
!python bench.py \
  --config configs/index/config_wiki_chroma_10k.json \
  --backend chroma \
  --index wiki_chroma_10k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/wiki_chroma_10k \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-29 05:26:40.822893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764394000.844278   52329 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764394000.850600   52329 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764394000.866441   52329 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764394000.866466   52329 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764394000.866470   52329 computation_placer.cc:177] computation placer alr

#### Fix for Wiki Chroma 10k

In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"


/content


#### delete the old 10k Chroma folder (broken DB)

In [ ]:
!rm -rf indices/wiki_chroma/wiki_chroma_10k


#### re-ingest 10k into Chroma

In [ ]:
!python scripts/chroma_ingest_2.py \
  --meta_path data/embeddings/wikipedia/wiki_meta_10000.parquet \
  --vec_path data/embeddings/wikipedia/wiki_embeddings_10000.npy \
  --persist_dir indices/wiki_chroma/wiki_chroma_10k \
  --collection wiki_chroma_10k


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Ingesting 10,000 records into 'wiki_chroma_10k' @ indices/wiki_chroma/wiki_chroma_10k
  0% 0/10 [00:00<?, ?it/s]Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given
100% 10/10 [00:14<00:00,  1.44s/it]
✓ Manifest created: indices/wiki_chroma/wiki_chroma_10k/wiki_chroma_10k_manifest.json
✓ Done.


#### quick check

In [ ]:
!ls indices/wiki_chroma/wiki_chroma_10k


chroma.sqlite3			      wiki_chroma_10k_manifest.json
e1c8fff3-7d6c-4e10-8046-89f12fefa23d


#### rerun the 10k Chroma + LLM bench

In [ ]:
!python bench.py \
  --config configs/index/config_wiki_chroma_10k.json \
  --backend chroma \
  --index wiki_chroma_10k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/wiki_chroma_10k \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-29 05:38:44.513319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764394724.537103   55676 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764394724.544251   55676 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764394724.560169   55676 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764394724.560193   55676 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764394724.560197   55676 computation_placer.cc:177] computation placer alr

#### Wiki Chroma 50k + LLM

In [ ]:
!rm -rf indices/wiki_chroma/wiki_chroma_50k



#### Re-run the 50k ingest

In [ ]:
!python scripts/chroma_ingest_2.py \
  --meta_path data/embeddings/wikipedia/wiki_meta_50000.parquet \
  --vec_path data/embeddings/wikipedia/wiki_embeddings_50000.npy \
  --persist_dir indices/wiki_chroma/wiki_chroma_50k \
  --collection wiki_chroma_50k


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Ingesting 50,000 records into 'wiki_chroma_50k' @ indices/wiki_chroma/wiki_chroma_50k
  0% 0/50 [00:00<?, ?it/s]Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given
100% 50/50 [01:50<00:00,  2.20s/it]
✓ Manifest created: indices/wiki_chroma/wiki_chroma_50k/wiki_chroma_50k_manifest.json
✓ Done.


#### Config for wiki_chroma_50k

In [ ]:
!python scripts/inject_chroma_config.py \
  --output_config configs/index/config_wiki_chroma_50k.json \
  --index_name wiki_chroma_50k \
  --persist_dir indices/wiki_chroma/wiki_chroma_50k \
  --collection_name wiki_chroma_50k


✓ Created new Chroma config: configs/index/config_wiki_chroma_50k.json
✓ Added index: wiki_chroma_50k
✓ active_index = wiki_chroma_50k


#### Run bench for Wiki 50k Chroma + LLM

In [ ]:
!python bench.py \
  --config configs/index/config_wiki_chroma_50k.json \
  --backend chroma \
  --index wiki_chroma_50k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/wiki_chroma_50k \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-29 06:33:13.450171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764397993.469775   71098 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764397993.475773   71098 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764397993.490982   71098 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764397993.491008   71098 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764397993.491011   71098 computation_placer.cc:177] computation placer alr

### MULTI-DOMAIN (10k & 30k)

#### I. REBUILD CHUNKS (MULTI-DOMAIN 10k & 30k)

#### Multi-domain 10k slice

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING


In [ ]:
!pip install -r requirements.txt --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/61.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [ ]:
!pip install chromadb --quiet


In [ ]:
!ls -R


.:
 bench.py			 data	    requirements.txt
 configs			 indices    results
'Copy of Preprocessing.ipynb'	 logs	    retrieval_runner.py
 Copy_of_Step2_Retrieval.ipynb	 pipeline   scripts

./configs:
index

./configs/index:
backup				    config_wiki_chroma_50k.json
config.json			    config_wiki_spark_10k.json
config_multidomain_chroma_50k.json  config_wiki_spark_30k.json
config_multidomain_spark_50k.json   config_wiki_spark_50k.json
config_mutidomain_faiss_50k.json    wiki_10k_config.json
config_tmp.json			    wiki_30k_config.json
config_wiki_chroma_10k.json	    wiki_50k_config.json
config_wiki_chroma_30k.json

./configs/index/backup:
config.json  config_v2_updatefaissmetapath.json

./data:
embeddings  processed  queries	raw

./data/embeddings:
multi_default	       pubmed		   wiki_50k_sharded  wiki_sharded_50k
multi_default_sharded  sample_vectors.npy  wikipedia	     wiki_single
multi_domain	       stackexchange	   wiki_sharded
multidomain_sharded    wiki_10k_sharded    wiki_sharded_10k

.

In [ ]:
!python scripts/embedding_slice.py \
  --emb_path data/embeddings/multi_default/multi_vectors.npy \
  --meta_path data/embeddings/multi_default/multi_meta.parquet \
  --out_dir data/embeddings/multi_domain \
  --n 10000 \
  --domain multidomain

Loading metadata: data/embeddings/multi_default/multi_meta.parquet
Metadata rows available: 79164
Slicing first 10000 metadata rows...
Saved: data/embeddings/multi_domain/multidomain_meta_10000.parquet
Loading embeddings: data/embeddings/multi_default/multi_vectors.npy
Embedding matrix shape: (79164, 384)
Slicing first 10000 embedding rows...
Saved: data/embeddings/multi_domain/multidomain_embeddings_10000.npy
Files created:
  - data/embeddings/multi_domain/multidomain_meta_10000.parquet
  - data/embeddings/multi_domain/multidomain_embeddings_10000.npy


#### Multi-domain 30k slices

In [ ]:
!python scripts/embedding_slice.py \
  --emb_path data/embeddings/multi_default/multi_vectors.npy \
  --meta_path data/embeddings/multi_default/multi_meta.parquet \
  --out_dir data/embeddings/multi_domain \
  --n 30000 \
  --domain multidomain


Loading metadata: data/embeddings/multi_default/multi_meta.parquet
Metadata rows available: 79164
Slicing first 30000 metadata rows...
Saved: data/embeddings/multi_domain/multidomain_meta_30000.parquet
Loading embeddings: data/embeddings/multi_default/multi_vectors.npy
Embedding matrix shape: (79164, 384)
Slicing first 30000 embedding rows...
Saved: data/embeddings/multi_domain/multidomain_embeddings_30000.npy
Files created:
  - data/embeddings/multi_domain/multidomain_meta_30000.parquet
  - data/embeddings/multi_domain/multidomain_embeddings_30000.npy


### Build FAISS (Multi-domain 10k)

In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

import numpy as np
import faiss
from pathlib import Path

# Correct 10k embeddings
emb_path = Path("data/embeddings/multi_domain/multidomain_embeddings_10000.npy")
vecs = np.load(emb_path)
print("Loaded embeddings:", vecs.shape)

d = vecs.shape[1]

print("Building 10k HNSW index...")
index = faiss.IndexHNSWFlat(d, 32)  # M=32 like other HNSW indices
index.hnsw.efConstruction = 200
index.add(vecs.astype("float32"))

print("Indexed vectors:", index.ntotal)

out_path = Path("indices/wikipedia/multidomain_faiss_single_10k.faiss")
out_path.parent.mkdir(parents=True, exist_ok=True)
faiss.write_index(index, str(out_path))

print(f"✅ Saved corrected 10k FAISS index to: {out_path}")


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Loaded embeddings: (10000, 384)
Building 10k HNSW index...
Indexed vectors: 10000
✅ Saved corrected 10k FAISS index to: indices/wikipedia/multidomain_faiss_single_10k.faiss


In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

import json
from pathlib import Path

cfg_path = Path("configs/index/config_mutidomain_faiss_10k.json")

with cfg_path.open() as f:
    cfg = json.load(f)

print("Before fix:\n", cfg, "\n")

idx_cfg = cfg["indices"]["multidomain_faiss_single_10k"]
idx_cfg["faiss_path"] = "indices/wikipedia/multidomain_faiss_single_10k.faiss"
idx_cfg["meta_path"] = "data/embeddings/multi_domain/multidomain_meta_10000.parquet"

with cfg_path.open("w") as f:
    json.dump(cfg, f, indent=2)

print("✅ After fix:\n", cfg)


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Before fix:
 {'indices': {'multidomain_faiss_single_10k': {'backend': 'faiss', 'faiss_path': 'indices/wikipedia/multidomain_faiss_single_10k.faiss', 'meta_path': 'data/embeddings/multi_domain/multidomain_meta_30000.parquet', 'model_name': 'BAAI/bge-small-en-v1.5', 'metric': 'ip', 'normalize_query': True, 'index_metric': 'ip', 'vector_normed': True, 'skip_path_checks': False}}} 

✅ After fix:
 {'indices': {'multidomain_faiss_single_10k': {'backend': 'faiss', 'faiss_path': 'indices/wikipedia/multidomain_faiss_single_10k.faiss', 'meta_path': 'data/embeddings/multi_domain/multidomain_meta_10000.parquet', 'model_name': 'BAAI/bge-small-en-v1.5', 'metric': 'ip', 'normalize_query': True, 'index_metric': 'ip', 'vector_normed': True, 'skip_path_checks': False}}}


In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

!python scripts/build_faiss_index.py \
  --subset_dir data/embeddings/multi_domain \
  --index_group wikipedia \
  --index_name multidomain_faiss_single_10k \
  --domain multidomain \
  --out_config configs/index/config_mutidomain_faiss_10k.json


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Metadata file: data/embeddings/multi_domain/multidomain_meta_30000.parquet
Embeddings file: data/embeddings/multi_domain/multidomain_embeddings_30000.npy
Metadata rows: 30000
Embedding matrix: (30000, 384)
Building FAISS HNSW index...
Indexed vectors: 30000
Saved FAISS index: indices/wikipedia/multidomain_faiss_single_10k.faiss
Config written to: configs/index/config_mutidomain_faiss_10k.json
Done.


### Build FAISS (Multi-domain 30k)

In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

!python scripts/build_faiss_index.py \
  --subset_dir data/embeddings/multi_domain \
  --index_group wikipedia \
  --index_name multidomain_faiss_single_30k \
  --domain multidomain \
  --out_config configs/index/config_mutidomain_faiss_30k.json


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Metadata file: data/embeddings/multi_domain/multidomain_meta_30000.parquet
Embeddings file: data/embeddings/multi_domain/multidomain_embeddings_30000.npy
Metadata rows: 30000
Embedding matrix: (30000, 384)
Building FAISS HNSW index...
Indexed vectors: 30000
Saved FAISS index: indices/wikipedia/multidomain_faiss_single_30k.faiss
Config written to: configs/index/config_mutidomain_faiss_30k.json
Done.


#### Correcting Multi-domain 10k

In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

import numpy as np
import faiss
from pathlib import Path

# Use the correct 10k embeddings
emb_path = Path("data/embeddings/multi_domain/multidomain_embeddings_10000.npy")
vecs = np.load(emb_path)
print("Loaded embeddings:", vecs.shape)

d = vecs.shape[1]

print("Building 10k HNSW index...")
index = faiss.IndexHNSWFlat(d, 32)  # same M=32 as other HNSW indices
index.hnsw.efConstruction = 200
index.add(vecs.astype("float32"))

print("Indexed vectors:", index.ntotal)

out_path = Path("indices/wikipedia/multidomain_faiss_single_10k.faiss")
out_path.parent.mkdir(parents=True, exist_ok=True)
faiss.write_index(index, str(out_path))

print(f"✅ Saved corrected 10k FAISS index to: {out_path}")


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Loaded embeddings: (10000, 384)
Building 10k HNSW index...
Indexed vectors: 10000
✅ Saved corrected 10k FAISS index to: indices/wikipedia/multidomain_faiss_single_10k.faiss


In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

import json
from pathlib import Path

cfg_path = Path("configs/index/config_mutidomain_faiss_10k.json")

with cfg_path.open() as f:
    cfg = json.load(f)

print("Before fix:\n", cfg, "\n")

idx_cfg = cfg["indices"]["multidomain_faiss_single_10k"]
idx_cfg["faiss_path"] = "indices/wikipedia/multidomain_faiss_single_10k.faiss"
idx_cfg["meta_path"] = "data/embeddings/multi_domain/multidomain_meta_10000.parquet"

with cfg_path.open("w") as f:
    json.dump(cfg, f, indent=2)

print("✅ After fix:\n", cfg)


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Before fix:
 {'indices': {'multidomain_faiss_single_10k': {'backend': 'faiss', 'faiss_path': 'indices/wikipedia/multidomain_faiss_single_10k.faiss', 'meta_path': 'data/embeddings/multi_domain/multidomain_meta_30000.parquet', 'model_name': 'BAAI/bge-small-en-v1.5', 'metric': 'ip', 'normalize_query': True, 'index_metric': 'ip', 'vector_normed': True, 'skip_path_checks': False}}} 

✅ After fix:
 {'indices': {'multidomain_faiss_single_10k': {'backend': 'faiss', 'faiss_path': 'indices/wikipedia/multidomain_faiss_single_10k.faiss', 'meta_path': 'data/embeddings/multi_domain/multidomain_meta_10000.parquet', 'model_name': 'BAAI/bge-small-en-v1.5', 'metric': 'ip', 'normalize_query': True, 'index_metric': 'ip', 'vector_normed': True, 'skip_path_checks': False}}}


#### Multi-domain 10k FAISS single (no LLM)

In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

!python bench.py \
  --config configs/index/config_mutidomain_faiss_10k.json \
  --index multidomain_faiss_single_10k \
  --backend faiss \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/multidomain_faiss_single_10k


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
2025-11-29 20:44:14.921466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764449054.940600   15442 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764449054.946563   15442 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764449054.962032   15442 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764449054.962059   15442 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more t

#### Multi-domain 10k FAISS + LLM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING"
!pwd
!ls


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
 bench.py			 data	    requirements.txt
 configs			 indices    results
'Copy of Preprocessing.ipynb'	 logs	    retrieval_runner.py
 Copy_of_Step2_Retrieval.ipynb	 pipeline   scripts


In [ ]:
!pip install -r requirements.txt


In [ ]:
!pip install chromadb


In [ ]:
!python bench.py \
  --config configs/index/config_mutidomain_faiss_10k.json \
  --index multidomain_faiss_single_10k \
  --backend faiss \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/multidomain_faiss_single_10k_llm \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-29 21:56:23.268950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764453383.302830    6706 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764453383.312722    6706 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764453383.340001    6706 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764453383.340032    6706 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764453383.340040    6706 computation_placer.cc:177] computation placer alr

#### Multi-domain 30k FAISS (no LLM)

In [ ]:
%cd "/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING"


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING


In [ ]:
!python bench.py \
  --config configs/index/config_mutidomain_faiss_30k.json \
  --index multidomain_faiss_single_30k \
  --backend faiss \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/multidomain_faiss_single_30k


2025-11-29 22:29:48.911744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764455388.944072   15133 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764455388.953607   15133 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764455388.977038   15133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764455388.977070   15133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764455388.977078   15133 computation_placer.cc:177] computation placer alr

#### Multi-domain 30k FAISS + LLM

In [ ]:
!python bench.py \
  --config configs/index/config_mutidomain_faiss_30k.json \
  --index multidomain_faiss_single_30k \
  --backend faiss \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/multidomain_faiss_single_30k_llm \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0



2025-11-29 22:30:16.233515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764455416.252665   15272 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764455416.258537   15272 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764455416.272977   15272 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764455416.273003   15272 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764455416.273007   15272 computation_placer.cc:177] computation placer alr

### Multidomain 10k – FAISS Sharded + LLM

#### Shard multidomain 10k

In [ ]:
%cd "/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING"

!python scripts/shard_index_v2.py \
  --meta_path data/embeddings/multi_domain/multidomain_meta_10000.parquet \
  --emb_path data/embeddings/multi_domain/multidomain_embeddings_10000.npy \
  --domain multidomain \
  --out_meta_dir data/embeddings/multidomain_sharded_10k \
  --out_vectors_dir data/embeddings/multidomain_sharded_10k \
  --out_index_dir indices/multidomain_sharded_10k \
  --out_manifest_dir indices/multidomain_sharded_10k \
  --n_shards 4 \
  --save_vectors 1


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Loaded 10,000 vectors (dim=384)
Normalizing vectors (L2)...
Sharding domain 'multidomain': 100% 4/4 [00:01<00:00,  2.23it/s]
Manifest written: indices/multidomain_sharded_10k/multidomain_10000_manifest.json
Total rows = 10000


#### Inject 10k sharded index into config

In [ ]:
!python scripts/inject_sharded_index.py \
  --config_path configs/index/config_mutidomain_faiss_10k.json \
  --index_name multidomain_faiss_sharded_10k \
  --faiss_dir indices/multidomain_sharded_10k \
  --meta_dir data/embeddings/multidomain_sharded_10k \
  --set_active



Successfully added FAISS-sharded entry to config json file
New index name: multidomain_faiss_sharded_10k
FAISS shards: 4
META shards: 4
Manifest: indices/multidomain_sharded_10k/multidomain_10000_manifest.json
Updated config: configs/index/config_mutidomain_faiss_10k.json


#### Run bench for multidomain 10k sharded + LLM

In [ ]:
!python bench.py \
  --config configs/index/config_mutidomain_faiss_10k.json \
  --backend faiss \
  --index multidomain_faiss_sharded_10k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 \
  --iters 3 \
  --warmup 1 \
  --results_dir results/multidomain_faiss_sharded_10k_llm \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-29 23:24:35.272368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764458675.292691   28836 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764458675.299093   28836 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764458675.314823   28836 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764458675.314849   28836 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764458675.314853   28836 computation_placer.cc:177] computation placer alr

### Multidomain 30k – FAISS Sharded + LLM

#### Shard multidomain 30k

In [ ]:
!python scripts/shard_index_v2.py \
  --meta_path data/embeddings/multi_domain/multidomain_meta_30000.parquet \
  --emb_path data/embeddings/multi_domain/multidomain_embeddings_30000.npy \
  --domain multidomain \
  --out_meta_dir data/embeddings/multidomain_sharded_30k \
  --out_vectors_dir data/embeddings/multidomain_sharded_30k \
  --out_index_dir indices/multidomain_sharded_30k \
  --out_manifest_dir indices/multidomain_sharded_30k \
  --n_shards 8 \
  --save_vectors 1


Loaded 30,000 vectors (dim=384)
Normalizing vectors (L2)...
Sharding domain 'multidomain': 100% 8/8 [00:07<00:00,  1.02it/s]
Manifest written: indices/multidomain_sharded_30k/multidomain_30000_manifest.json
Total rows = 30000


#### Inject 30k sharded index into config

In [ ]:
!python scripts/inject_sharded_index.py \
  --config_path configs/index/config_mutidomain_faiss_30k.json \
  --index_name multidomain_faiss_sharded_30k \
  --faiss_dir indices/multidomain_sharded_30k \
  --meta_dir data/embeddings/multidomain_sharded_30k \
  --set_active



Successfully added FAISS-sharded entry to config json file
New index name: multidomain_faiss_sharded_30k
FAISS shards: 8
META shards: 8
Manifest: indices/multidomain_sharded_30k/multidomain_30000_manifest.json
Updated config: configs/index/config_mutidomain_faiss_30k.json


#### Run bench for multidomain 30k sharded + LLM

In [ ]:
!python bench.py \
  --config configs/index/config_mutidomain_faiss_30k.json \
  --backend faiss \
  --index multidomain_faiss_sharded_30k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 \
  --iters 3 \
  --warmup 1 \
  --results_dir results/multidomain_faiss_sharded_30k_llm \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-29 23:57:22.314181: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764460642.333738   37033 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764460642.339859   37033 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764460642.355263   37033 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764460642.355289   37033 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764460642.355293   37033 computation_placer.cc:177] computation placer alr

### Multidomain Chroma 10k

#### Ingest 10k into Chroma

In [ ]:
%cd "/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING"

!python scripts/chroma_ingest_2.py \
  --meta_path data/embeddings/multi_domain/multidomain_meta_10000.parquet \
  --vec_path data/embeddings/multi_domain/multidomain_embeddings_10000.npy \
  --persist_dir indices/multidomain_chroma/multidomain_chroma_10k \
  --collection multidomain_chroma_10k


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Ingesting 10,000 records into 'multidomain_chroma_10k' @ indices/multidomain_chroma/multidomain_chroma_10k
  0% 0/10 [00:00<?, ?it/s]Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given
100% 10/10 [00:17<00:00,  1.77s/it]
✓ Manifest created: indices/multidomain_chroma/multidomain_chroma_10k/multidomain_chroma_10k_manifest.json
✓ Done.


#### Create config for 10k Chroma

In [ ]:
!python scripts/inject_chroma_config.py \
  --output_config configs/index/config_multidomain_chroma_10k.json \
  --index_name multidomain_chroma_10k \
  --persist_dir indices/multidomain_chroma/multidomain_chroma_10k \
  --collection_name multidomain_chroma_10k


✓ Created new Chroma config: configs/index/config_multidomain_chroma_10k.json
✓ Added index: multidomain_chroma_10k
✓ active_index = multidomain_chroma_10k


#### Bench multidomain Chroma 10k + LLM

In [ ]:
!python bench.py \
  --config configs/index/config_multidomain_chroma_10k.json \
  --backend chroma \
  --index multidomain_chroma_10k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/multidomain_chroma_10k \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-30 00:58:58.457387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764464338.478114   53066 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764464338.484411   53066 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764464338.500078   53066 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764464338.500103   53066 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764464338.500107   53066 computation_placer.cc:177] computation placer alr

### Multidomain Chroma 30k

#### Ingest 30k into Chroma

In [ ]:
%cd "/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING"

!python scripts/chroma_ingest_2.py \
  --meta_path data/embeddings/multi_domain/multidomain_meta_30000.parquet \
  --vec_path data/embeddings/multi_domain/multidomain_embeddings_30000.npy \
  --persist_dir indices/multidomain_chroma/multidomain_chroma_30k \
  --collection multidomain_chroma_30k


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Ingesting 30,000 records into 'multidomain_chroma_30k' @ indices/multidomain_chroma/multidomain_chroma_30k
  0% 0/30 [00:00<?, ?it/s]Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given
100% 30/30 [01:28<00:00,  2.95s/it]
✓ Manifest created: indices/multidomain_chroma/multidomain_chroma_30k/multidomain_chroma_30k_manifest.json
✓ Done.


#### Create config for 30k Chroma

In [ ]:
!python scripts/inject_chroma_config.py \
  --output_config configs/index/config_multidomain_chroma_30k.json \
  --index_name multidomain_chroma_30k \
  --persist_dir indices/multidomain_chroma/multidomain_chroma_30k \
  --collection_name multidomain_chroma_30k


✓ Created new Chroma config: configs/index/config_multidomain_chroma_30k.json
✓ Added index: multidomain_chroma_30k
✓ active_index = multidomain_chroma_30k


#### Bench multidomain Chroma 30k + LLM

In [ ]:
!python bench.py \
  --config configs/index/config_multidomain_chroma_30k.json \
  --backend chroma \
  --index multidomain_chroma_30k \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/multidomain_chroma_30k \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-30 01:45:37.770205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764467137.790131   65694 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764467137.796194   65694 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764467137.813854   65694 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764467137.813880   65694 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764467137.813884   65694 computation_placer.cc:177] computation placer alr

### MULTIDOMAIN SPARK 10K

#### Create Spark config for 10k

In [ ]:
%cd "/content/drive/MyDrive/AMS 560 PROJECT_SCALING"

!python scripts/inject_spark_config.py \
  --index_key multidomain_spark_10000 \
  --manifest_path indices/multidomain_sharded_10k/multidomain_10000_manifest.json \
  --out_path configs/index/config_multidomain_spark_10k.json


/content/drive/.shortcut-targets-by-id/1LXinasMHpowO__Kbw9zFlXFvFH8GZA00/AMS 560 PROJECT_SCALING
✓ Spark config.json created
  index_key     : multidomain_spark_10000
  manifest_path : indices/multidomain_sharded_10k/multidomain_10000_manifest.json
  out_path      : configs/index/config_multidomain_spark_10k.json


#### Run Spark experiment + LLM (10k)

In [ ]:
!python bench.py \
  --config configs/index/config_multidomain_spark_10k.json \
  --backend spark \
  --index multidomain_spark_10000 \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/multidomain_spark_10k_llm \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-30 03:29:50.782916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764473390.803237   91754 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764473390.809409   91754 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764473390.825155   91754 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764473390.825181   91754 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764473390.825185   91754 computation_placer.cc:177] computation placer alr

### MULTIDOMAIN SPARK 30K

#### Create Spark config for 30k

In [ ]:
!python scripts/inject_spark_config.py \
  --index_key multidomain_spark_30000 \
  --manifest_path indices/multidomain_sharded_30k/multidomain_30000_manifest.json \
  --out_path configs/index/config_multidomain_spark_30k.json


✓ Spark config.json created
  index_key     : multidomain_spark_30000
  manifest_path : indices/multidomain_sharded_30k/multidomain_30000_manifest.json
  out_path      : configs/index/config_multidomain_spark_30k.json


#### Run Spark experiment + LLM (30k)

In [ ]:
!python bench.py \
  --config configs/index/config_multidomain_spark_30k.json \
  --backend spark \
  --index multidomain_spark_30000 \
  --qfile data/queries/all_sources_eval.txt \
  --top_k 3 --iters 3 --warmup 1 \
  --results_dir results/multidomain_spark_30k_llm \
  --enable_llm \
  --llm_model TinyLlama/TinyLlama-1.1B-Chat-v1.0


2025-11-30 04:17:49.937318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764476269.957241  108345 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764476269.963322  108345 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764476269.978547  108345 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764476269.978573  108345 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764476269.978577  108345 computation_placer.cc:177] computation placer alr